In [1]:
import requests
import json
import time
import datetime
from helpers.history_fetcher import HistoryFetcher
import dateutil.parser
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

## Analysis of countries stability <--> wikipedia changes

In [2]:
def wiki_change_factor(wiki_name,year_start, outlier_factor):
    
    # fetch the changes
    history_fetcher = HistoryFetcher(wiki_name)
    response = history_fetcher.get_history(str(year_start)+'0101000000', '20171128000000')
    dates = list(map(lambda revision: revision['timestamp'], response))
    dates_pd = pd.DataFrame(dates, columns=['date'])
    dates_pd['change']= 1;
    
    # aggregate per moth
    changes_aggregated_month = np.zeros([(2017 - year_start)*12+nr_filt_window,1])
    date_last = datetime.date(year=year_start,month=11,day=1)
    for y in range(year_start,2017):
        for m in range(0,12):
            date_current = datetime.date(year=y,month=m+1,day=1)
            changes_month = np.sum(dates_pd[ dates_pd['date'] > date_last][ dates_pd['date'] < date_current]['change'])
            changes_aggregated_month[-1+m+(y-year_start)*12]=changes_month
            date_last = date_current
    
    changes_aggregated_month = np.squeeze(changes_aggregated_month)
    #plt.bar(range(len(changes_aggregated_month)), changes_aggregated_month)
    #plt.title('wiki changes/month for' + wiki_name)
    #plt.show()
    
     # select outliers
    nr_outliers = np.sum(changes_aggregated_month[changes_aggregated_month > outlier_factor* np.mean(changes_aggregated_month)])
    sum_all = np.sum(changes_aggregated_month)
    
    return nr_outliers/sum_all

In [3]:
countries = ['Ukraine','Poland','Spain', 'Netherlands','France','Germany','Switzerland']
for cntry in countries:
    print(cntry, wiki_change_factor(cntry, 2010, outlier_factor = 1.5))

/Users/patryk/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:16: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


Ukraine 0.527371578553
Poland 0.438545688546
Spain 0.490640653836
Netherlands 0.257175660161
France 0.4624
Germany 0.469008264463
Switzerland 0.319691119691
